In [ ]:
%reset -f
import ipyparallel
import os
rc = ipyparallel.Client(profile="mpi")
view = rc[:]
view.apply(os.chdir, os.getcwd())
view['stop'] = True
print view

view.block=True

In [ ]:
%%px --block
from mpi4py import MPI
mpi = MPI.COMM_WORLD
bcast = mpi.bcast
barrier = mpi.barrier
rank = mpi.rank


# Engines Imports
import sys
#!pwd   # Replace sys.path.append with this path
sys.path.append('/home/user/JorgeMijares/air-water-vv/2d/Interactive/Temp_Flume_GUI')

import proteus
## Required imports
from proteus.iproteus import * 
from proteus.mprans import SpatialTools as st
from petsc4py import PETSc
from threading import Thread
import tempFlume as plant
import tempFlume_so as plant_so
from proteus import default_n, default_s, default_so, Comm, Context
print "MPI rank: %i/%i" % (mpi.rank,mpi.size)

In [ ]:
# Only Kernel Imports

import pandas as pd
from IPython.display import clear_output

# Kernel and Engines imports
#with view.sync_imports():  

    

## Simulation Parameters

In [ ]:

#Parameters
dt=0.1            # Not sure if disabled, more testing needed, assign same value as dtOut too
cfl=0.75          # Not working yet, modify too in tempflume.py
dtOut=0.1          # Modify nDTout
T=0.2           # Total Simulation Time
Tank_he=0.03        # Domain he
Caisson_he=0.03     # Interpolation distance of caisson perimeter
enSphere=False        # Replace Caisson with sphere for testing of parameters
radius=0.1


#Select Case, if for loop deactivated
index  = 44             #Refer to DoE_TempFlume.xlsx to check case conditions
scale  = 21

## Monitor Functions

In [2]:
import numpy as np
def mpi_order(seq):
    """Return elements of a sequence ordered by MPI rank.

    The input sequence is assumed to be ordered by engine ID."""
    ranks = view['rank']
    rank_indices = np.argsort(ranks)
    return [seq[x] for x in rank_indices]


showTriangles = True
showAir = False


def plot_current_results():
    global t_0, plt0
    """Makes a blocking call to retrieve remote data and displays the solution mesh
    as a contour plot.

    Parameters
    ----------
    in_place : bool
        By default it calls clear_output so that new plots replace old ones.  Set
        to False to allow keeping of all previous outputs.
    """
    import numpy as np
    import matplotlib.tri as mtri
    import math
    global nn, x, y, vof, triangles, t, phi, u, v, cfl, p, dt
    view.apply_sync(load_simulation_globals)
    x = np.concatenate(mpi_order(view['x']))
    y = np.concatenate(mpi_order(view['y']))
    u = np.concatenate(mpi_order(view['u']))
    v = np.concatenate(mpi_order(view['v']))
    vof = np.concatenate(mpi_order(view['vof']))
    shifts = np.cumsum([0] + mpi_order(view['nn'])[:-1])
    flat_triangles = np.concatenate([tri + shift for tri, shift in zip(mpi_order(view['triangles']), shifts)])

    t = mpi_order(view['t'])[0]
    phi = np.concatenate(mpi_order(view['phi']))
    cfl = mpi_order(view['cfl'])[0]
    p = np.concatenate(mpi_order(view['p']))
    dt = mpi_order(view['dt'])[0]

    Vmax = np.amax(np.sqrt(u[:] ** 2 + v[:] ** 2))
    print "u_max={0:.3f}, v_max={1:.3f}, Vmax={2:.3f}, cfl={3:.3f}, dt={4:.5f}".format(np.amax(u[:]),
                                                                                       np.amax(v[:]),
                                                                                       Vmax,
                                                                                       np.asarray(cfl).max() * dt,
                                                                                       dt)                                                                                       

def load_simulation_globals():
    """Put some variables we need in engine namespace.

    These can then be retrieved by clients for inspection, visualization, etc.
    """
    global nn, x, y, vof, triangles, t, phi, u, v, cfl, p, dt
    model_vof = ns.modelList[1].levelModelList[-1]
    model_ls = ns.modelList[2].levelModelList[-1]
    # save solution and grid data for plotting purposes
    cfl = ns.modelList[0].levelModelList[-1].q[('cfl', 0)]
    dt = ns.systemStepController.dt_system
    x = ns.modelList[0].levelModelList[-1].mesh.nodeArray[:, 0]
    y = ns.modelList[0].levelModelList[-1].mesh.nodeArray[:, 1]
    triangles = ns.modelList[0].levelModelList[-1].mesh.elementNodesArray
    p = ns.modelList[0].levelModelList[-1].u[0].dof
    u = ns.modelList[0].levelModelList[-1].u[1].dof
    v = ns.modelList[0].levelModelList[-1].u[2].dof
    vof = ns.modelList[1].levelModelList[-1].u[0].dof_last
    phi = ns.modelList[2].levelModelList[-1].u[0].dof_last
    nn = len(x)
    # print "p={0}, u={1}, v={2}, triangles={3}, vof={4}, phi={5}".format(len(p),len(u),len(v),len(triangles),len(vof),len(phi))
    t = ns.systemStepController.t_system
    cfl *= dt


def simulation_alive():
    """Return True if the simulation thread is still running on any engine.
    """
    return any(view.apply_sync(lambda: simulation_thread.is_alive()))

def wait_alive():
    """Return True if the simulation thread is still running on any engine.
    """
    
    import datetime as dt, time
    while not all(view.apply_sync(lambda: simulation_thread.is_alive())):
        print 'Waiting all threads to start...'
        time.sleep(5)
    print "All threads ready"



def monitor_simulation(refresh=5.0):
    global t_0
    """Monitor the simulation progress and call plotting routine.

    Supress KeyboardInterrupt exception if interrupted, ensure that the last 
    figure is always displayed and provide basic timing and simulation status.

    Parameters
    ----------
    refresh : float
      Refresh interval between calls to retrieve and plot data.  The default
      is 5s, adjust depending on the desired refresh rate, but be aware that 
      very short intervals will start having a significant impact.

    """

    import datetime as dt, time
    wait_alive()
    if not simulation_alive():
        plot_current_results()
        print 'Simulation has already finished, no monitoring to do.'
        error = True
        return error
    t_0 = 0.
    t0 = dt.datetime.now()
    try:
        while simulation_alive():
            plot_current_results()
            error = False
            tmon = dt.datetime.now() - t0
            t_sim = view.apply_sync(lambda: ns.systemStepController.t_system)[0]
            print 'Monitored for: %s. at t=%12.5e' % (tmon, t_sim)
            time.sleep(refresh)  # so we don't hammer the server too fast
        print "All threads down"
    except (KeyboardInterrupt):  # , error.TimeoutError):
        msg = 'Monitoring interrupted, simulation is ongoing!'
    else:
        t_sim = view.apply_sync(lambda: ns.systemStepController.t_system)[0]
        if t_sim < T:
            msg = "\x1b[31mStep Failed at t=%12.5e \x1b[0m" % (t_sim)
            error = True
        else:
            msg = 'Simulation completed!'

    tmon = dt.datetime.now() - t0
    print msg
    print 'Monitored for: %s.' % tmon
    return error




## Simulation Setup

In [ ]:

#Experimental Case

Scales = [21,25]        # Scale could be 21 or 25

for s_ind, scale in enumerate(Scales):
#if True:
        xl    = pd.ExcelFile("DoE_TempFlume.xlsx")
        df1   = xl.parse(0)
        df2   = xl.parse(1)
        df3   = xl.parse(2)
        df4   = xl.parse(3, header=0, index_col=0)
        df5   = xl.parse(4, header=0, index_col=0)
        

        clear_output()

        if scale == 21:
            df_DoE  = df4
            df_data = df2

        elif scale==25:
            df_DoE  = df5
            df_data = df3
        else:
            print 'Wrong Scale'
            () + 1
        for index in range(df4.shape[0]):
            #if True:
            #view['stop']=True

            clear_output()
            test_name     = 'Results/S{0}T{1}'.format(scale,index)
            status        = df_DoE['Finished'].tolist()[index]
            if status == 'Completed':
                continue
            
            # Read Index from DoE table
            iFlow   = df_DoE['Flow Speed'].tolist()[index]
            iDepth  = df_DoE['Depth'].tolist()[index]
            iOffset = df_DoE['Offset'].tolist()[index]
            
            # Read Tag parameters from table
            tagFlow   = df_data['Speed Tags'].tolist()[iFlow]
            tagDepth  = df_data['Depth Tags'].tolist()[iDepth]
            tagOffset = df_data['Offset Tags'].tolist()[iOffset]
            
            # Read parameters from table
            flowSpeed  = df_data['Flow Speed'].tolist()[iFlow]
            waterLevel = df_data['Depth'].tolist()[iDepth]
            offset     = df_data['Offset'].tolist()[iOffset]
            
            #Change Context
            
            def Reload_Context():
                    reload(Context)
                    reload(plant)
                    reload(plant_so)
                    
            view.apply_sync(Reload_Context)
            view['plant.opts.water_level'] = waterLevel
            view['plant.opts.tank_dim']    = (3.0,waterLevel+0.5)

            # plant.opts.wind_velocity=(flowSpeed,0.)
            view['plant.opts.inflow_velocity'] = flowSpeed
            # plant.opts.outflow_velocity=flowSpeed
            view['plant.opts.caisson_scale']   = float(scale)
            view['plant.opts.caisson_Yoffset'] = -offset
            
            view['plant.opts.dtOut']        = dtOut
            view['plant.opts.sphere']       = enSphere
            view['plant.opts.sphereRadius'] = radius
            view['plant.opts.dt_fixed']     = dt
            view['plant.opts.cfl']          = cfl
            view['plant.opts.T']            = T
            view['plant.opts.he']           = Tank_he
            view['plant.opts.he_caisson']   = Caisson_he

            print 'Scale={0}, Test={1}'.format(scale,index)
            print 'No Scaled values:'
            print 'Speed(ft/s)={0}, Depth(ft)={1}, Weigth(lb)={2}'.format(
                                                tagFlow,
                                                tagDepth,
                                                tagOffset)
            print 'Simulated Values:'
            print 'Speed={0}, Depth={1}, Offset={2}'.format(
                                                flowSpeed,
                                                waterLevel,
                                                -offset)
            def Setup_Simulation():
                global ct,ns, simulation_thread
                
                plant.Update_Model()
                Context.setFromModule(plant,mutable=True)
                
                plant_so.ct = Context.get()
                
                so = plant_so
                so.tnList = [0.0]+[i*plant.opts.dtOut for i in range(1,plant.nDTout+1)]        
                info = open("TimeList.txt","w")
                for time in so.tnList:
                    info.write(str(time)+"\n")
                info.close()
                plant_so.tnList=so.tnList
                pList=[]
                nList=[]
                so.sList=[]
                OptDB = PETSc.Options()
                for (p,n) in so.pnList:
                    so.sList.append(default_s)
                    pList.append(__import__(p))
                    reload(pList[-1])
                    nList.append(__import__(n))
                    reload(nList[-1])
                    pList[-1].name = p
                    nList[-1].multilevelLinearSolver = default_n.KSP_petsc4py
                    nList[-1].levelLinearSolver = default_n.KSP_petsc4py
                    OptDB.setValue(nList[-1].linear_solver_options_prefix+"ksp_type", "preonly")
                    OptDB.setValue(nList[-1].linear_solver_options_prefix+"pc_type", "lu")
                    OptDB.setValue(nList[-1].linear_solver_options_prefix+"pc_factor_mat_solver_package","superlu_dist")
                opts.save_dof = True
                opts.dataDir='.'
                opts.probDir='.'
                opts.logLevel=7
                opts.verbose=True
                opts.viewMesh=True


                ## Create numerical solution
                ns = NumericalSolution.NS_base(so, pList, nList, so.sList, opts)


                ## Start Simulation
                simulation_thread = Thread(target = lambda : ns.calculateSolution('run1'))
                simulation_thread.start()
                
            pr_list=view.apply_sync(Setup_Simulation)
            print 'Simulation Started'
            
            
            ## Monitor Output
            error=monitor_simulation(refresh=5.0)      
            
            ## Export/Save Data
            import os   
            print test_name
            
            test_name='Results/S{0}T{1}'.format(scale,index)
            checkpath=os.path.exists('./'+test_name) 
            if not checkpath:
                    os.mkdir(test_name)
            with open(test_name+'/opts.txt', 'w+') as fp:
                opts = view.apply_sync(lambda : plant.opts._asdict().items())[0]
                fp.write('\n'.join('%s %s' % (key , val) for key, val in opts))

            !cp tempFlume_p.xmf ./{test_name}/tempFlume_p.xmf
            !cp tempFlume_p.h5  ./{test_name}/tempFlume_p.h5
            !cp forceHistory_p.txt ./{test_name}/forceHistory_p.txt
            !cp forceHistory_v.txt  ./{test_name}/forceHistory_v.txt
            !cp TimeList.txt ./{test_name}/TimeList.txt
            if not error:
                Exp_Status='Completed'
            else:
                Exp_Status='Crashed'

            print Exp_Status

            # Create a Pandas Excel writer using XlsxWriter as the engine.
            writer = pd.ExcelWriter('DoE_TempFlume.xlsx', engine='xlsxwriter')

            if scale==21:
                df4.loc[index,'Finished']=Exp_Status
                df4.loc[index,'Time']=T
            elif scale==25:
                df5.loc[index,'Finished']=Exp_Status
                df5.loc[index,'Time']=T
                print 'Scale Error'

            # Convert the dataframe to an XlsxWriter Excel object.
            df1.to_excel(writer, sheet_name='Parameters')
            df2.to_excel(writer, sheet_name='Parameters Scale 1-21')
            df3.to_excel(writer, sheet_name='Parameters Scale 1-25')
            df4.to_excel(writer, sheet_name='Tests Scale 1-21')
            df5.to_excel(writer, sheet_name='Tests Scale 1-25')
            # Close the Pandas Excel writer and output the Excel file.
            writer.save()
            print "Stop all process"
            view.wait(pr_list)
            

In [ ]:
# error = False

# if not error:
#     Exp_Status='Completed'
# else:
#     Exp_Status='Crashed'

# print Exp_Status
# writer = pd.ExcelWriter('DoE_TempFlume.xlsx', engine='xlsxwriter')

# if scale==21:
#     df4.loc[index,'Finished']=Exp_Status
#     df4.loc[index,'Time']=T
# elif scale==25:
#     df5.loc[index,'Finished']=Exp_Status
#     df5.loc[index,'Time']=T
#     print 'Scale Error'

# # Convert the dataframe to an XlsxWriter Excel object.
# df1.to_excel(writer, sheet_name='Parameters')
# df2.to_excel(writer, sheet_name='Parameters Scale 1-21')
# df3.to_excel(writer, sheet_name='Parameters Scale 1-25')
# df4.to_excel(writer, sheet_name='Tests Scale 1-21')
# df5.to_excel(writer, sheet_name='Tests Scale 1-25')
# # Close the Pandas Excel writer and output the Excel file.
# writer.save()